## Call Tools to provide more accurate responses

In [ ]:
from langchain.agents import Tool
from langchain.agents import tool
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent

In [ ]:
@tool
def add_number(inputs: str) :
    """
    Tool: add_numbers
    Description: Add numeric values extracted from text input
    Input: str (e.g. "Add numbers 10, 20 and 30")
    Output: dict (e.g. {"result": 60})
    """
    numbers = [int(x) for x in inputs.replace(",", "").split() if x.isdigit()]
    result = sum(numbers)
    return str(result)

In [ ]:
# add_tool = Tool(
#     name="AddTool",
#     func=add_number,
#     description="Add a list of numbers and return the result",
# )
# print("Tool Object:", add_tool)


In [ ]:
test_input = "What is the sum of 10, 20 and 30"
print(add_number.invoke(test_input))

test_input = "What is the sum of ten, 20 and 30"
print(add_number.invoke(test_input))

In [ ]:
model = init_chat_model("openai:gpt-4.1", temperature=0)
agent = create_react_agent(
    tools=[add_number],
    model=model,
)


In [ ]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "What is the sum of 10, 20 and 30"}]}
)
for message in result["messages"]:
    print(message.content)


In [ ]:
from functools import reduce

def multiply_number(inputs: str) :
    """
    Tool: multiply_numbers
    Description: Multiply numeric values extracted from text input
    Input: str (e.g. "Multiply numbers 10, 20 and 30")
    Output: dict (e.g. {"result": 60})
    """
    numbers = [int(x) for x in inputs.replace(",", "").split() if x.isdigit()]
    if not numbers:
        return "No numbers found in input"
    result = reduce(lambda x, y: x * y, numbers)
    return str(result)

def divide_number(inputs: str) :
    """
    Tool: divide_numbers
    Description: Divide numeric values extracted from text input
    Input: str (e.g. "Divide numbers 10, 20 and 30")
    Output: dict (e.g. {"result": 60})
    """
    numbers = [int(x) for x in inputs.replace(",", "").split() if x.isdigit()]
    if not numbers:
        return "No numbers found in input"
    result = reduce(lambda x, y: x / y, numbers)
    return str(result)

In [ ]:
model = init_chat_model("openai:gpt-4.1", temperature=0)
agent = create_react_agent(
    tools=[add_number, multiply_number, divide_number],
    model=model,
    prompt= "You are a helpful math assistant that can perform addition, multiplication and division. Use the tools to answer the question. If you don't know the answer, just say that you don't know, don't try to make up an answer. Also show your reasoning step by step.",
)
result = agent.invoke(
    {"messages": [{"role": "user", "content": "What is the sum of 10, 20 and 30 multiplied by 2 divided by 3?"}]}
)
for message in result["messages"]:
    print(message.content)


In [ ]:
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun, WikipediaAPIWrapper
from langchain_community.tools.google_finance.tool import GoogleFinanceQueryRun, GoogleFinanceAPIWrapper
from langchain_community.tools.golden_query.tool import GoldenQueryRun, GoldenQueryAPIWrapper
from langchain_community.tools.openweathermap.tool import OpenWeatherMapQueryRun, OpenWeatherMapAPIWrapper
wiki_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
google_finance_tool = GoogleFinanceQueryRun(api_wrapper=GoogleFinanceAPIWrapper())
golden_query_tool = GoldenQueryRun(api_wrapper=GoldenQueryAPIWrapper())
openweather_tool = OpenWeatherMapQueryRun(api_wrapper=OpenWeatherMapAPIWrapper())
model = init_chat_model("openai:gpt-4.1", temperature=0)
agent = create_react_agent(
    tools=[add_number, multiply_number, divide_number, wiki_tool, google_finance_tool, golden_query_tool, openweather_tool],
    model=model,
    prompt= "You are a helpful assistant that can perform various tasks. Use the tools to answer the question. If you don't know the answer, just say that you don't know, don't try to make up an answer. Also show your reasoning step by step, and cite any sources your used"
)

In [ ]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "What is Bell's inequality"},
                  {"role": "user", "content": "What is the current price of SPY ETF"},
                  {"role": "user", "content": "What is the weather in Seattle tomorrow"}]}
)
for message in result["messages"]:
    print(message.content)

In [1]:
from langchain_community.tools.google_finance import GoogleFinanceQueryRun
from langchain_community.utilities.google_finance import GoogleFinanceAPIWrapper

tool = GoogleFinanceQueryRun(api_wrapper=GoogleFinanceAPIWrapper())

In [2]:
tool.run("Google")

'\nQuery: Google\nNo summary information\n'

In [3]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")
from langchain_community.agent_toolkits.load_tools import load_tools

tools = load_tools(["google-scholar", "google-finance"], llm=llm)
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llm, tools)

events = agent.stream(
    {"messages": [("user", "What is Google's stock?")]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is Google's stock?
================================== Ai Message ==================================
Tool Calls:
  google_finance (call_0unChfv8sQSbQOivTDN5KCLo)
 Call ID: call_0unChfv8sQSbQOivTDN5KCLo
  Args:
    query: Alphabet Inc.
================================= Tool Message =================================
Name: google_finance


Query: Alphabet Inc.
No summary information

================================== Ai Message ==================================

It seems that I wasn't able to retrieve specific information about Google's stock (Alphabet Inc.) at this time. You can check stock prices and other financial details on financial news websites or stock market platforms for the latest information.
